# Import

In [1]:
# Spotify
import spotipy
import sys
from spotipy.oauth2 import SpotifyClientCredentials

# Analysis/ NLP
import colorama
from colorama import Fore
import textblob
from textblob import TextBlob

# Visualization/ Output
from tabulate import tabulate
import csv
import requests
from PIL import Image
from io import BytesIO

# Initialize Spotify API

In [5]:
# Credentials | localhost:8888/callback
# Might need to export + echo @ bash

# CLIENT_SECRET HAS BEEN ROTATED, STORE @ GITIGNORE
# CLIENT_ID = "X"
# CLIENT_SECRET = "X"

client_credentials_manager = SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

## Artist Info + Fetch Top N Tracks (1st Iteration)

In [6]:
def get_artist_info(artist_name):

    # Enter artist's name
    results = sp.search(q='artist:' + artist_name, type='artist')

    if results['artists']['items']:
        artist = results['artists']['items'][0]
        print(f'Artist: {artist["name"]}')
        print(f'Popularity: {artist["popularity"]}')

        # Top N tracks --> default: 10 songs
        top_tracks = sp.artist_top_tracks(artist['id'])

        print('Top Tracks:')
        for track in top_tracks['tracks']:
            print(f'- {track["name"]}')

        # Discography audio features
        for track in top_tracks['tracks']:

            # https://developer.spotify.com/documentation/web-api/reference/get-audio-features
            audio_features = sp.audio_features(track['id'])[0]

            print(f'Audio Features of "{track["name"]}":')

            # [0, 1] --> 1 is high confidence track is acoustic
            print(f'- Acousticness: {audio_features["acousticness"]}')

            # [0, 1] --> 0 is least danceable, 1 is most danceable
            print(f'- Danceability: {audio_features["danceability"]}')

            # duration of track in ms
            print(f'- Duration: {audio_features["duration_ms"]}')

            # [0, 1] --> perceptual measure of intensity and activity 
            # Perceptual features --> (e.g. dynamic range, perceived loudness, timbre...)
            print(f'- Energy: {audio_features["energy"]}')

            # [0, 1] --> confidence interval where values close to 1.0 have a greater likelihood of containing no vocal content
            print(f'Instrumentalness: {audio_features["instrumentalness"]}')

            # Loudness of track in dB, average across the entire track
            print(f'Loudness: {audio_features["loudness"]}')

            # Presence of spoken words in track, values closer to 1.0 have more speech
            print(f'Speechiness: {audio_features["speechiness"]}')

            # Tempo of track in BPM
            print(f'Tempo: {audio_features["tempo"]}')

            # Sentiment analysis of the song through music positivity
            # [0, 1] --> High valence is more positive, low valence is more negative
            print(f'Valence: {audio_features["valence"]}')

    else:
        print(f'Artist "{artist_name}" not found.')

# Prompt
artist_name = input('Enter an artist\'s name: ')
get_artist_info(artist_name)

Artist: Taylor Swift
Popularity: 100
Top Tracks:
- Cruel Summer
- Blank Space
- Style
- Anti-Hero
- august
- cardigan
- Lover
- Don’t Blame Me
- Karma
- Shake It Off
Audio Features of "Cruel Summer":
- Acousticness: 0.117
- Danceability: 0.552
- Duration: 178427
- Energy: 0.702
Instrumentalness: 2.06e-05
Loudness: -5.707
Speechiness: 0.157
Tempo: 169.994
Valence: 0.564
Audio Features of "Blank Space":
- Acousticness: 0.085
- Danceability: 0.753
- Duration: 231827
- Energy: 0.678
Instrumentalness: 1.64e-06
Loudness: -5.421
Speechiness: 0.0644
Tempo: 96.006
Valence: 0.583
Audio Features of "Style":
- Acousticness: 0.00253
- Danceability: 0.598
- Duration: 231000
- Energy: 0.786
Instrumentalness: 0.0016
Loudness: -5.572
Speechiness: 0.0383
Tempo: 95.019
Valence: 0.456
Audio Features of "Anti-Hero":
- Acousticness: 0.13
- Danceability: 0.637
- Duration: 200690
- Energy: 0.643
Instrumentalness: 1.8e-06
Loudness: -6.571
Speechiness: 0.0519
Tempo: 97.008
Valence: 0.533
Audio Features of "augu

## Output w/ Tabulate (2nd Iteration)

In [ ]:
def get_artist_info_tabulate(artist_name):

    # Enter artist's name
    results = sp.search(q='artist:' + artist_name, type='artist')

    if results['artists']['items']:

        artist = results['artists']['items'][0]

        artist_info = {
            "Artist": artist["name"],
            "Popularity": artist["popularity"]
        }

        # Top N tracks --> default: 10 songs
        # Store info to list
        top_tracks = sp.artist_top_tracks(artist['id'])
        top_tracks_info = []

        # Discography audio features
        for track in top_tracks['tracks']:

            # # https://developer.spotify.com/documentation/web-api/reference/get-audio-features
            audio_features = sp.audio_features(track['id'])[0]
            track_info = {
                "Track Name": track["name"],
                "Acousticness": audio_features["acousticness"],
                "Danceability": audio_features["danceability"],
                "Duration": audio_features["duration_ms"],
                "Energy": audio_features["energy"],
                "Instrumentalness": audio_features["instrumentalness"],
                "Loudness": audio_features["loudness"],
                "Speechiness": audio_features["speechiness"],
                "Tempo": audio_features["tempo"],
                "Valence": audio_features["valence"]
            }

            # List for tabulate
            top_tracks_info.append(track_info)

        print(tabulate([artist_info], headers="keys", tablefmt="pretty"))
        print(tabulate(top_tracks_info, headers="keys", tablefmt="pretty"))

    else:
        print(f'Artist "{artist_name}" not found.')

# Prompt
artist_name = input('Enter an artist\'s name: ')
get_artist_info_tabulate(artist_name)


## Output w/ CSV (3rd Iteration)

In [ ]:
def get_artist_info_csv(artist_name):

    # Enter artist's name
    results = sp.search(q='artist:' + artist_name, type='artist')

    if results['artists']['items']:

        artist = results['artists']['items'][0]

        # Top N tracks --> default: 10 songs
        top_tracks = sp.artist_top_tracks(artist['id'])

        # https://www.freecodecamp.org/news/with-open-in-python-with-statement-syntax-example/
        # https://www.geeksforgeeks.org/how-to-open-a-file-using-the-with-statement/
        # https://note.nkmk.me/en/python-file-io-open-with/
        with open(f'{artist_name}_info.csv', 'w', newline='') as csvfile:
            fieldnames = ["Track Name", "Acousticness", "Danceability", "Duration", "Energy", "Instrumentalness", "Loudness", "Speechiness", "Tempo", "Valence"]
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()

            # # # https://developer.spotify.com/documentation/web-api/reference/get-audio-features
            for track in top_tracks['tracks']:

                audio_features = sp.audio_features(track['id'])[0]
                writer.writerow({
                    "Track Name": track["name"],
                    "Acousticness": audio_features["acousticness"],
                    "Danceability": audio_features["danceability"],
                    "Duration": audio_features["duration_ms"],
                    "Energy": audio_features["energy"],
                    "Instrumentalness": audio_features["instrumentalness"],
                    "Loudness": audio_features["loudness"],
                    "Speechiness": audio_features["speechiness"],
                    "Tempo": audio_features["tempo"],
                    "Valence": audio_features["valence"]
                })

        print(f'Artist: {artist["name"]}')
        print(f'Info saved @ {artist_name}_info.csv')

    else:
        print(f'Artist "{artist_name}" not found.')

# Prompt
artist_name = input('Enter an artist\'s name: ')
get_artist_info_csv(artist_name)

## Artist's Face (1st Iteration)

In [ ]:
def get_artist_face(artist_name):

    # Enter artist's name
    results = sp.search(q='artist:' + artist_name, type='artist')

    if results['artists']['items']:

        artist = results['artists']['items'][0]

        # Face
        if artist['images']:

            image_url = artist['images'][0]['url']
            response = requests.get(image_url)

            # https://developer.spotify.com/documentation/web-api/reference/get-an-artist
            if response.status_code == 200:
                
                image = Image.open(BytesIO(response.content))
                image.save(f'{artist_name}_image.jpg')

                print(f'Artist\'s Name: {artist["name"]}')
                print(f'Artist\'s face saved as {artist_name}_face.jpg')

            else:
                print('Failed to fetch artist\'s profile image.')

        else:
            print(f'Artist "{artist_name}" does not have an image of their face.')

    else:
        print(f'Artist "{artist_name}" not found.')

# Prompt
artist_name = input('Enter an artist\'s name: ')
get_artist_face(artist_name)